In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import pickle as pk

import tensorflow.keras
from tensorflow.keras.models import load_model
    
test_data_path = "test_data.pk"
MODEL_NAME = "model.keras"

model = load_model(MODEL_NAME)

2023-12-04 13:58:08.989409: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 13:58:08.989433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 13:58:08.990176: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 13:58:08.995956: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 13:58:09.808027: W tensorflow/compiler/tf2

In [2]:
import pickle as pk

test_data = pk.load(open(test_data_path, "rb"))
X_test = test_data["X"]
X_test_oh = test_data["X_oh"]
alphabet_size = test_data["alphabet_size"]

y_test = test_data["y"]
event_mapping = test_data["event_mapping"]

len(X_test), len(X_test_oh), len(y_test)

(227925, 227925, 227925)

In [3]:
event_mapping

{'24': 0,
 '40': 1,
 '61': 2,
 '72': 3,
 '41': 4,
 '75': 5,
 '69': 6,
 '39': 7,
 '83': 8,
 '74': 9,
 '109': 10,
 '27': 11,
 '105': 12,
 '34': 13,
 '111': 14,
 '98': 15,
 '85': 16,
 '87': 17,
 '116': 18,
 '108': 19,
 '101': 20,
 '63': 21,
 '<EOS>': 22,
 '62': 23,
 '42': 24,
 '96': 25,
 '86': 26,
 '57': 27,
 '95': 28,
 '58': 29,
 '13': 30,
 '21': 31,
 '112': 32,
 '80': 33,
 '32': 34,
 '45': 35,
 '118': 36,
 '36': 37,
 '30': 38,
 '104': 39,
 '51': 40,
 '102': 41,
 '64': 42,
 '53': 43,
 '89': 44,
 '50': 45,
 '17': 46,
 '47': 47,
 '28': 48,
 '106': 49,
 '29': 50,
 '73': 51,
 '46': 52,
 '120': 53,
 '16': 54,
 '82': 55,
 '67': 56,
 '54': 57,
 '107': 58,
 '76': 59,
 '60': 60,
 '49': 61,
 '84': 62,
 '100': 63,
 '14': 64,
 '119': 65,
 '12': 66,
 '68': 67,
 '78': 68,
 '52': 69,
 '91': 70,
 '23': 71,
 '90': 72,
 '18': 73,
 '35': 74,
 '117': 75,
 '115': 76,
 '20': 77,
 '25': 78,
 '<SOS>': 79,
 '56': 80,
 '130': 81,
 '93': 82,
 '94': 83,
 '79': 84,
 '65': 85,
 '113': 86,
 '43': 87,
 '97': 88,
 '38':

In [4]:
sample_idx = 5
test = model.predict(np.array(X_test[sample_idx]))

1/1 [==============================] - 0s 269ms/step


In [5]:
idx = 6
np.argmax(test[idx]), np.argmax(X_test_oh[sample_idx][idx])

(22, 77)

In [6]:
#X_test[5]
y_test[sample_idx]

1

In [7]:
#def data_generator():
#    global X_test
    
#    idx = 0
#    while idx < len(X_test):
#        yield np.array(X_test[idx])
#        idx += 1

#data_gen = data_generator()

#X_pred = model.predict(data_gen, steps=len(X_test))

In [8]:
X_pred = list()
for idx, seq in enumerate(X_test):
    current_prediction = None
    for i in range(len(seq)):
        pred = model(np.array(seq[:i+1]))
        if current_prediction is None:
            current_prediction = pred
        else:
            current_prediction = np.vstack((current_prediction, pred))
    X_pred.append(current_prediction)
    if idx % 500 == 0:
        print(idx)
    if idx == 7000:
        break

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000


In [9]:
print(len(X_pred))
X_pred = X_pred[:-1]
print(len(X_pred))

7001
7000


In [10]:
import pickle as pk

pk.dump(X_pred, open("X_pred_full.pk", "wb"))